# Image Classification Using Feature Extractors (CatsDogsPanda dataset)

* [About VGG16](#aboutVgg)
* [Load VGG Pre Trained Weights](#vgg16)
* [Load the Images](#loadImages)
* [Extract Features](#extractFeatures)
* [Build Classifier](#buildClassifier)
* [Evaluate Model](#evaluate)

**Objective:** Extract the features from the images of CatsDogsPanda and classify using Logistic Regression.

## How to extract the features from the images?

Use any convolutional network architecture but before flattening them and feeding to the Fully connected network to classify, chop of the Fully connected network, which results in the features which are extracted from the trained images.

<a id='aboutVgg'> </a>
## About VGG16

 * Refer to Paper: [VERY DEEP CONVOLUTIONAL NETWORKS FOR LARGE-SCALE IMAGE RECOGNITION](https://arxiv.org/pdf/1409.1556.pdf)

 * Input Image size: 224x224 RGB Images
 * Input Image (224,224,3) -> Conv3(kernel:64) -> maxPool -> Conv3(128)-> maxPool -> Conv3(256) -> maxPool -> Conv3(512) -> maxPool -> Conv3(512) -> maxPool (7x7x512) -> FC

<a id='vgg16'></a>
## Load VGG Pre Trained Weights
* Exclude the Fully connected Layers

In [1]:
from keras.applications import VGG16

Using Theano backend.


In [2]:
# load VGG16 network
model = VGG16(weights='imagenet', include_top=False)

In [3]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

<a id='loadImages'></a>
## Load the Images

In [4]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pathlib
import glob
import random
import os

In [5]:
print('numpy: ', np.__version__)
print('opencv: ', cv2.__version__)

numpy:  1.14.2
opencv:  3.4.4


In [6]:
ImageDir = '../datasets/animals'

# recursively go over the datset folder and get the file names.
ImageList = list(pathlib.Path(ImageDir).rglob('*.jpg'))
random.shuffle(ImageList)
print(len(ImageList))

3000


In [7]:
## load all the images
## resize them to one size
def preprocess_image(img, width, height, interpolation=cv2.INTER_AREA):
    return( cv2.resize(img, (width, height), interpolation))

In [8]:
# load all images.
# need to resize the image to 224x224
def load_all_images(imageList, verbose=-1):
    data = []
    labels = []

    # go over each image - read them and extract class label from the path
    for (i, imagepath) in enumerate(imageList):
        filename = imagepath.as_posix()
        img = cv2.imread(filename)
        label = filename.split(os.path.sep)[-2]
        img = preprocess_image(img, 224, 224)

        data.append(img)
        labels.append(label)

        if ( verbose > 0 and i > 0 and (i+1) % verbose == 0):
            print('Info: Processed {} / {}'.format(i+1, len(imageList)))


    return(np.array(data), np.array(labels))

In [9]:
##
print('Load All Images...')
# for test
#(data, labels) = load_all_images(ImageList[0:32], verbose=250)

# all images
(data, labels) = load_all_images(ImageList, verbose=250)

Load All Images...
Info: Processed 250 / 3000
Info: Processed 500 / 3000
Info: Processed 750 / 3000
Info: Processed 1000 / 3000
Info: Processed 1250 / 3000
Info: Processed 1500 / 3000
Info: Processed 1750 / 3000
Info: Processed 2000 / 3000
Info: Processed 2250 / 3000
Info: Processed 2500 / 3000
Info: Processed 2750 / 3000
Info: Processed 3000 / 3000


In [10]:
print(data.shape)

(3000, 224, 224, 3)


<a id='extractFeatures'></a>
## Extract Features

In [11]:
?model.predict

In [12]:
features = model.predict(data)

In [13]:
features.shape

(3000, 7, 7, 512)

In [14]:
# flatten the feature vector after the maxPooling2D outputs
features = features.reshape(features.shape[0], 512*7*7)

In [15]:
print(labels[0:3])

['dogs' 'dogs' 'dogs']


### Convert the labels

In [16]:
from sklearn.preprocessing import LabelEncoder

In [17]:
le = LabelEncoder()

In [18]:
labels = le.fit_transform(labels)

In [19]:
print(labels[0:3])

[1 1 1]


In [20]:
print(len(labels))

3000


<a id='classifier'></a>

## Build Classifier

In [21]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

In [22]:
?GridSearchCV

In [23]:
?LogisticRegression

In [24]:
#  Params: 
#    C - Inverse of regularization strength [ small value specify stronger regularization]
#LogisticRegression()
print("Tuning hyperparameters...")
params = {"C": [0.1, 1.0, 10.0, 100.0, 1000.0, 10000.0]}
classifier = GridSearchCV(LogisticRegression(), params, cv=3)

Tuning hyperparameters...


In [25]:
# the data is already shuffled
# so use 75 percent for  training and remaining for testing.
split_idx = int(len(data) * 0.75)

In [26]:
split_idx

2250

### Fit the model

In [27]:
print(len(features[:split_idx]))
print(len(labels[:split_idx]))

2250
2250


In [28]:
classifier.fit(features[:split_idx], labels[:split_idx])

/usr/local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/usr/local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/usr/local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed

GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'C': [0.1, 1.0, 10.0, 100.0, 1000.0, 10000.0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [29]:
 print("Info: Best hyperparameters: {}".format(classifier.best_params_))

Info: Best hyperparameters: {'C': 0.1}


<a id='evaluate'></a>

## Evaluate the model

In [30]:
from sklearn.metrics import confusion_matrix, classification_report #metric

In [31]:
pred = classifier.predict(features[split_idx:])
print(classification_report(labels[split_idx:], pred, target_names=["cat", "dog", "panda"]))

              precision    recall  f1-score   support

         cat       0.95      0.99      0.97       250
         dog       0.99      0.94      0.96       256
       panda       0.98      0.98      0.98       244

   micro avg       0.97      0.97      0.97       750
   macro avg       0.97      0.97      0.97       750
weighted avg       0.97      0.97      0.97       750



In [32]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(labels[split_idx:], pred))

[[248   1   1]
 [ 11 241   4]
 [  2   2 240]]


In [33]:
from sklearn.metrics import accuracy_score
print(accuracy_score(labels[split_idx:], pred))

0.972


In [36]:
import pickle

In [38]:
## Save the model
f = open('CatsDogsPanda_FeatureExtractor_model', "wb")
f.write(pickle.dumps(classifier.best_estimator_))
f.close()

In [39]:
classifier.best_estimator_

LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [44]:
type(features)

numpy.ndarray

In [46]:
np.save('CatsDogsPanda_Features', features)

In [48]:
feature_test = np.load('CatsDogsPanda_Features.npy')

In [50]:
features == feature_test

array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])